http://spark.apache.org/

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important; font-size:100%;}</style>"))
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
from pprint import pprint
import datetime
import pandas as pd
import numpy as np

In [3]:
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
session_name = "sales_data"

spark = SparkSession.builder.appName(session_name).getOrCreate()

spark

In [5]:
N = 35_000

current_date = datetime.date.today()
current_time = datetime.datetime.now()

pval = [.05,.25,.3,.05,.1,.1,.1,.05]

np.random.RandomState(seed=7)

df = pd.DataFrame.from_dict(
    {
        'trans_dates':np.random.choice(np.arange('2010-01-01', '2021-01-01', dtype='datetime64[D]'), size=N),
        'LOCATIONS':np.random.choice(['CA','TX','NY','OH', 'FL'], size=(N,), p=[.1,.2,.3,.35,.05]),
        'employees':np.random.choice(['Aiden', 'Leslie', 'Ian', 'Harlan', 'Maeve', 'Kendra', 'Allen', 'Jo'], size=(N,), p=pval),        
        'Sales/Hrs':np.random.choice(np.arange(1, 17, 1), size=N),
        'sales tot$':np.random.normal(loc=1000, scale=20, size=N),
        'doc_date':current_date.strftime("%m/%d/%Y"),
    })

df['actuals'] = np.where(df['trans_dates'] <= current_date.strftime("%Y-%m-%d"), True, False)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   trans_dates  35000 non-null  datetime64[ns]
 1   LOCATIONS    35000 non-null  object        
 2   employees    35000 non-null  object        
 3   Sales/Hrs    35000 non-null  int64         
 4   sales tot$   35000 non-null  float64       
 5   doc_date     35000 non-null  object        
 6   actuals      35000 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.6+ MB


In [6]:
df.head()

,trans_dates,LOCATIONS,employees,Sales/Hrs,sales tot$,doc_date,actuals
0,2013-08-10,TX,Leslie,13,999.850663,12/24/2021,True
1,2011-11-13,NY,Leslie,2,1014.180269,12/24/2021,True
2,2014-01-20,NY,Leslie,6,995.953437,12/24/2021,True
3,2011-02-23,NY,Leslie,14,1015.830561,12/24/2021,True
4,2019-12-13,OH,Aiden,7,1039.087062,12/24/2021,True


In [7]:
df['sales tot$'].sum()

34998321.30343107

In [8]:
def clean_cols(x=str):
    x = x.lower().replace(",","").replace(",","").replace(":","_").replace("$","").replace("-","").replace("/","_")
    x = x.replace(" ","_")
    x = x.replace("___","_")
    x = x.replace("__","_")
    return x

In [9]:
df.columns

Index(['trans_dates', 'LOCATIONS', 'employees', 'Sales/Hrs', 'sales tot$',
       'doc_date', 'actuals'],
      dtype='object')

In [10]:
df.columns = list(map(clean_cols, df.columns))
df.columns

Index(['trans_dates', 'locations', 'employees', 'sales_hrs', 'sales_tot',
       'doc_date', 'actuals'],
      dtype='object')

In [11]:
sql_types = {
    'object':'string',
    'int64':'integer',
    'int32':'integer',
    'int16':'integer',
    'float64':'double',
    'float32':'double',
    'datetime64[ns]':'date'
}

for i, x in enumerate(df.dtypes):
    col_name = df.dtypes.index[i]
    col_type = str(df.dtypes.values[i])
    
    if 'date' in col_name:
        print(col_name, 'date', end='')
    else:
        print(col_name, sql_types.get(col_type, 'string'), end='')
    print(',')

trans_dates date,
locations string,
employees string,
sales_hrs integer,
sales_tot double,
doc_date date,
actuals string,


In [12]:
df.to_parquet("sales_data")

In [13]:
ds = spark.read.parquet("sales_data") 

In [14]:
ds.printSchema()

root
 |-- trans_dates: timestamp (nullable = true)
 |-- locations: string (nullable = true)
 |-- employees: string (nullable = true)
 |-- sales_hrs: long (nullable = true)
 |-- sales_tot: double (nullable = true)
 |-- doc_date: string (nullable = true)
 |-- actuals: boolean (nullable = true)



In [15]:
ds.show(truncate=False)

+-------------------+---------+---------+---------+------------------+----------+-------+
|trans_dates        |locations|employees|sales_hrs|sales_tot         |doc_date  |actuals|
+-------------------+---------+---------+---------+------------------+----------+-------+
|2013-08-09 20:00:00|TX       |Leslie   |13       |999.8506632992458 |12/24/2021|true   |
|2011-11-12 19:00:00|NY       |Leslie   |2        |1014.1802692686405|12/24/2021|true   |
|2014-01-19 19:00:00|NY       |Leslie   |6        |995.9534365891643 |12/24/2021|true   |
|2011-02-22 19:00:00|NY       |Leslie   |14       |1015.8305608573602|12/24/2021|true   |
|2019-12-12 19:00:00|OH       |Aiden    |7        |1039.087062120683 |12/24/2021|true   |
|2014-08-04 20:00:00|NY       |Ian      |13       |977.2432583966678 |12/24/2021|true   |
|2010-07-11 20:00:00|TX       |Ian      |4        |996.5728371254975 |12/24/2021|true   |
|2018-08-10 20:00:00|TX       |Allen    |13       |1009.2394739956186|12/24/2021|true   |
|2020-11-0

In [16]:
ds.dtypes

[('trans_dates', 'timestamp'),
 ('locations', 'string'),
 ('employees', 'string'),
 ('sales_hrs', 'bigint'),
 ('sales_tot', 'double'),
 ('doc_date', 'string'),
 ('actuals', 'boolean')]

In [17]:
ds.columns

['trans_dates',
 'locations',
 'employees',
 'sales_hrs',
 'sales_tot',
 'doc_date',
 'actuals']

In [18]:
ds = ds.withColumnRenamed('sales_tot', 'sales_total')

In [19]:
ds = ds.withColumn('doc_date', to_timestamp(col('doc_date'), "m/d/yyyy"))
ds.show()

+-------------------+---------+---------+---------+------------------+-------------------+-------+
|        trans_dates|locations|employees|sales_hrs|       sales_total|           doc_date|actuals|
+-------------------+---------+---------+---------+------------------+-------------------+-------+
|2013-08-09 20:00:00|       TX|   Leslie|       13| 999.8506632992458|2021-01-24 00:12:00|   true|
|2011-11-12 19:00:00|       NY|   Leslie|        2|1014.1802692686405|2021-01-24 00:12:00|   true|
|2014-01-19 19:00:00|       NY|   Leslie|        6| 995.9534365891643|2021-01-24 00:12:00|   true|
|2011-02-22 19:00:00|       NY|   Leslie|       14|1015.8305608573602|2021-01-24 00:12:00|   true|
|2019-12-12 19:00:00|       OH|    Aiden|        7| 1039.087062120683|2021-01-24 00:12:00|   true|
|2014-08-04 20:00:00|       NY|      Ian|       13| 977.2432583966678|2021-01-24 00:12:00|   true|
|2010-07-11 20:00:00|       TX|      Ian|        4| 996.5728371254975|2021-01-24 00:12:00|   true|
|2018-08-1

In [20]:
current_date = datetime.datetime.now()
current_date

datetime.datetime(2021, 12, 24, 9, 20, 53, 999747)

In [21]:
ds = ds.withColumn('current_date', lit(current_date))
ds.show(10, False)

+-------------------+---------+---------+---------+------------------+-------------------+-------+--------------------------+
|trans_dates        |locations|employees|sales_hrs|sales_total       |doc_date           |actuals|current_date              |
+-------------------+---------+---------+---------+------------------+-------------------+-------+--------------------------+
|2013-08-09 20:00:00|TX       |Leslie   |13       |999.8506632992458 |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2011-11-12 19:00:00|NY       |Leslie   |2        |1014.1802692686405|2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2014-01-19 19:00:00|NY       |Leslie   |6        |995.9534365891643 |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2011-02-22 19:00:00|NY       |Leslie   |14       |1015.8305608573602|2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2019-12-12 19:00:00|OH       |Aiden    |7        |1039.087062120683 |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.

In [22]:
ds = ds.withColumn('sales_total', round(col('sales_total') ,2))
ds.show(10, False)

+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------------+
|trans_dates        |locations|employees|sales_hrs|sales_total|doc_date           |actuals|current_date              |
+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------------+
|2013-08-09 20:00:00|TX       |Leslie   |13       |999.85     |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2011-11-12 19:00:00|NY       |Leslie   |2        |1014.18    |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2014-01-19 19:00:00|NY       |Leslie   |6        |995.95     |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2011-02-22 19:00:00|NY       |Leslie   |14       |1015.83    |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2019-12-12 19:00:00|OH       |Aiden    |7        |1039.09    |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|
|2014-08-04 20:00:00|NY       |Ian      |13     

In [23]:
ds.printSchema()

root
 |-- trans_dates: timestamp (nullable = true)
 |-- locations: string (nullable = true)
 |-- employees: string (nullable = true)
 |-- sales_hrs: long (nullable = true)
 |-- sales_total: double (nullable = true)
 |-- doc_date: timestamp (nullable = true)
 |-- actuals: boolean (nullable = true)
 |-- current_date: timestamp (nullable = false)



In [24]:
ds.printSchema()

root
 |-- trans_dates: timestamp (nullable = true)
 |-- locations: string (nullable = true)
 |-- employees: string (nullable = true)
 |-- sales_hrs: long (nullable = true)
 |-- sales_total: double (nullable = true)
 |-- doc_date: timestamp (nullable = true)
 |-- actuals: boolean (nullable = true)
 |-- current_date: timestamp (nullable = false)



In [25]:
def emp_sum(e):
    if e == "Maeve": return 'Group1'
    elif e == "Harlan": return 'Group2'
    else: return 'Group3'
    
emp_sum = udf(emp_sum, StringType())

In [26]:
ds = ds.withColumn('employee_summary', emp_sum('employees'))

In [27]:
ds.dtypes

[('trans_dates', 'timestamp'),
 ('locations', 'string'),
 ('employees', 'string'),
 ('sales_hrs', 'bigint'),
 ('sales_total', 'double'),
 ('doc_date', 'timestamp'),
 ('actuals', 'boolean'),
 ('current_date', 'timestamp'),
 ('employee_summary', 'string')]

In [28]:
ds.count()

35000

In [29]:
ds.show(10, False)

+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------------+----------------+
|trans_dates        |locations|employees|sales_hrs|sales_total|doc_date           |actuals|current_date              |employee_summary|
+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------------+----------------+
|2013-08-09 20:00:00|TX       |Leslie   |13       |999.85     |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|Group3          |
|2011-11-12 19:00:00|NY       |Leslie   |2        |1014.18    |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|Group3          |
|2014-01-19 19:00:00|NY       |Leslie   |6        |995.95     |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|Group3          |
|2011-02-22 19:00:00|NY       |Leslie   |14       |1015.83    |2021-01-24 00:12:00|true   |2021-12-24 09:20:53.999747|Group3          |
|2019-12-12 19:00:00|OH       |Aiden    |7      

In [30]:
ds.sample(False, fraction=0.1).show()

+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|        trans_dates|locations|employees|sales_hrs|sales_total|           doc_date|actuals|        current_date|employee_summary|
+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|2017-08-21 20:00:00|       CA|    Allen|        5|     995.74|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2018-04-08 20:00:00|       OH|      Ian|        5|     988.84|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2017-07-08 20:00:00|       NY|   Leslie|        8|     999.22|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2019-11-12 19:00:00|       OH|   Kendra|       14|    1007.88|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2012-11-11 19:00:00|       NY|    Aiden|        4|    1010.25|2021-01-24 00:12:00|   true

In [31]:
ds.sort('trans_dates').show()

+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|        trans_dates|locations|employees|sales_hrs|sales_total|           doc_date|actuals|        current_date|employee_summary|
+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|2009-12-31 19:00:00|       NY|   Leslie|        1|     977.27|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2009-12-31 19:00:00|       NY|      Ian|       11|    1022.62|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2009-12-31 19:00:00|       OH|    Maeve|       12|     1010.0|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group1|
|2009-12-31 19:00:00|       NY|      Ian|       16|     981.46|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2009-12-31 19:00:00|       CA|      Ian|        6|    1035.54|2021-01-24 00:12:00|   true

In [32]:
ds.sort('trans_dates').show()

+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|        trans_dates|locations|employees|sales_hrs|sales_total|           doc_date|actuals|        current_date|employee_summary|
+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|2009-12-31 19:00:00|       NY|   Leslie|        1|     977.27|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2009-12-31 19:00:00|       NY|      Ian|       11|    1022.62|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2009-12-31 19:00:00|       OH|    Maeve|       12|     1010.0|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group1|
|2009-12-31 19:00:00|       NY|      Ian|       16|     981.46|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2009-12-31 19:00:00|       CA|      Ian|        6|    1035.54|2021-01-24 00:12:00|   true

In [33]:
ds.filter(ds['locations']=='CA').show()

+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|        trans_dates|locations|employees|sales_hrs|sales_total|           doc_date|actuals|        current_date|employee_summary|
+-------------------+---------+---------+---------+-----------+-------------------+-------+--------------------+----------------+
|2020-11-02 19:00:00|       CA|       Jo|       10|    1025.57|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2010-11-05 20:00:00|       CA|   Leslie|        5|    1037.14|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2013-06-08 20:00:00|       CA|    Allen|        2|    1013.84|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2017-08-21 20:00:00|       CA|    Allen|        5|     995.74|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|
|2016-04-28 20:00:00|       CA|   Harlan|       10|     991.18|2021-01-24 00:12:00|   true

In [34]:
ds.groupBy('employee_summary').agg({'sales_total':'sum'}).show()

+----------------+-------------------+
|employee_summary|   sum(sales_total)|
+----------------+-------------------+
|          Group2| 1772455.9299999974|
|          Group1| 3465959.7900000056|
|          Group3|2.975990571999999E7|
+----------------+-------------------+



In [35]:
groups = ['locations']
aggs = ['sales_total']

func = [sum, mean, max, min]

expr = [f(col(c)) for f in func for c in aggs]

ds.groupBy(*groups).agg(*expr).orderBy('locations').show()

+---------+--------------------+-----------------+----------------+----------------+
|locations|    sum(sales_total)| avg(sales_total)|max(sales_total)|min(sales_total)|
+---------+--------------------+-----------------+----------------+----------------+
|       CA|   3506486.089999995| 999.853461648131|         1076.21|          940.41|
|       FL|  1766399.1800000002|999.6599773627619|         1081.21|          925.58|
|       NY|1.0556957949999992E7|999.9960168608499|         1077.27|           926.6|
|       OH|1.2187589740000056E7|999.9663390219935|         1084.59|           905.5|
|       TX|   6980888.479999987|999.9840252112859|         1084.31|          931.61|
+---------+--------------------+-----------------+----------------+----------------+



In [36]:
ds.groupBy('locations').pivot('employees').agg(sum('sales_total')).orderBy('locations').show()

+---------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|locations|            Aiden|             Allen|            Harlan|               Ian|                Jo|            Kendra|            Leslie|             Maeve|
+---------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|       CA|157878.2699999999| 352987.6400000001|179701.07000000004| 1063717.550000001|185838.13000000018| 332761.8600000001| 864463.3499999995|369138.22000000003|
|       FL|99035.19000000002| 174583.2999999999| 83769.36000000003| 534227.2599999998| 77997.46000000002|181933.23000000013| 438674.8300000002|176178.55000000005|
|       NY|533981.1100000003|1057267.3299999996| 568648.9199999993|3211283.0799999977| 513552.6699999999|1010777.8500000004| 2609025.640000002|1052421.3499999992|
|       OH|564429.1100

In [37]:
ds.filter(ds['employees']=='Aiden').groupBy('locations').pivot('actuals').agg(sum('sales_total')).orderBy('locations').show()

+---------+-----------------+
|locations|             true|
+---------+-----------------+
|       CA|157878.2699999999|
|       FL|99035.19000000002|
|       NY|533981.1100000003|
|       OH|564429.1100000003|
|       TX|328787.2000000003|
+---------+-----------------+



In [38]:
sc = spark.sparkContext

In [39]:

locations = (
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
    'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
    'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
    'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
    'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR', 'GU', 'VI', 'AS', 'MP',
    'DC'
)

stname = (
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
    'Kentucky', 'Louisiana',  'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
    'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
    'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
    'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming',
    'Puerto Rico', 'Guam', 'U.S. Virgin Islands', 'American Samoa', 'Northern Mariana Islands',
    'District of Columbia'
)

region = (
    'South', 'West', 'West', 'South', 'West', 'West', 'Northeast', 'South', 'South', 'South',
    'West', 'West', 'Midwest', 'Midwest', 'Midwest', 'Midwest', 'South', 'South', 'Northeast',
    'South', 'Northeast', 'Midwest', 'Midwest', 'South', 'Midwest', 'West', 'Midwest', 'West',
    'Northeast', 'Northeast', 'West', 'Northeast', 'South', 'Midwest', 'Midwest', 'South',
    'West', 'Northeast', 'Northeast', 'South', 'Midwest', 'South', 'South', 'West', 'Northeast',
    'South', 'West', 'South', 'Midwest', 'West', 'South', 'South', 'South', 'South', 'South',
    'South'
)

dt = sc.parallelize([Row(locations=col[0], stname=col[1], region=col[2]) for col in list(zip(locations,stname,region))]).toDF()

dt.show()

+---------+-----------+---------+
|locations|     stname|   region|
+---------+-----------+---------+
|       AL|    Alabama|    South|
|       AK|     Alaska|     West|
|       AZ|    Arizona|     West|
|       AR|   Arkansas|    South|
|       CA| California|     West|
|       CO|   Colorado|     West|
|       CT|Connecticut|Northeast|
|       DE|   Delaware|    South|
|       FL|    Florida|    South|
|       GA|    Georgia|    South|
|       HI|     Hawaii|     West|
|       ID|      Idaho|     West|
|       IL|   Illinois|  Midwest|
|       IN|    Indiana|  Midwest|
|       IA|       Iowa|  Midwest|
|       KS|     Kansas|  Midwest|
|       KY|   Kentucky|    South|
|       LA|  Louisiana|    South|
|       ME|      Maine|Northeast|
|       MD|   Maryland|    South|
+---------+-----------+---------+
only showing top 20 rows



In [40]:
dt.where("region == 'South'").select('locations', 'stname').show()

+---------+-------------------+
|locations|             stname|
+---------+-------------------+
|       AL|            Alabama|
|       AR|           Arkansas|
|       DE|           Delaware|
|       FL|            Florida|
|       GA|            Georgia|
|       KY|           Kentucky|
|       LA|          Louisiana|
|       MD|           Maryland|
|       MS|        Mississippi|
|       NC|     North Carolina|
|       OK|           Oklahoma|
|       SC|     South Carolina|
|       TN|          Tennessee|
|       TX|              Texas|
|       VA|           Virginia|
|       WV|      West Virginia|
|       PR|        Puerto Rico|
|       GU|               Guam|
|       VI|U.S. Virgin Islands|
|       AS|     American Samoa|
+---------+-------------------+
only showing top 20 rows



In [41]:
dj = ds.join(dt, on='locations', how='left')
dj.show()

+---------+-------------------+---------+---------+-----------+-------------------+-------+--------------------+----------------+----------+---------+
|locations|        trans_dates|employees|sales_hrs|sales_total|           doc_date|actuals|        current_date|employee_summary|    stname|   region|
+---------+-------------------+---------+---------+-----------+-------------------+-------+--------------------+----------------+----------+---------+
|       CA|2020-11-02 19:00:00|       Jo|       10|    1025.57|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|California|     West|
|       CA|2010-11-05 20:00:00|   Leslie|        5|    1037.14|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|California|     West|
|       OH|2019-12-12 19:00:00|    Aiden|        7|    1039.09|2021-01-24 00:12:00|   true|2021-12-24 09:20:...|          Group3|      Ohio|  Midwest|
|       OH|2011-12-31 19:00:00|    Aiden|        2|     973.25|2021-01-24 00:12:00|   true|202

In [ ]:
dj.createOrReplaceTempView("sales_data")

In [ ]:
spark.sql("SELECT * FROM sales_data LIMIT 10").show()

In [ ]:
spark.sql("SELECT count(*) AS total_count, round(sum(sales_total)/1000) AS total_sales_k FROM sales_data").show()

In [ ]:
spark.sql("SELECT region, round(sum(sales_total)/1000) AS sales_k FROM sales_data GROUP BY region ORDER BY 2 DESC").show()

In [ ]:
pdf = spark.sql("SELECT region, stname, employees, sales_hrs, sales_total FROM sales_data WHERE region IN ('Midwest', 'West')").toPandas()
pdf.head()

In [ ]:
pdf['region'].value_counts(dropna=False)